In [1]:
!pip install sentence-transformers pandas pinecone-client

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable


# Pinecone and Dataset

In [4]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import logging
import yfinance as yf

C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
# Initialize Pinecone
pc = Pinecone(api_key="09bb980b-6cef-48c3-9aa5-63f3cbc9885e")
index = pc.Index("financial-data-index")

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['C:\\Users\\Pham Ty\\AppData\\Roaming\\Python\\Python312\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone


In [7]:
# Initialize SentenceTransformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
logger.info("SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:__main__:SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.


In [8]:
# Folder containing the CSV files
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'

# Folder to save the processed CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'

In [9]:
# Định nghĩa danh sách các mã chứng khoán
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
    'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
    'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
    'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
    'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
    'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
    'CMG', 'BCS', 'UAA']

In [10]:
def fetch_and_save_stock_data(symbol, start_date='2019-01-01', end_date='2024-01-01'):
    """
    Fetch stock data from Yahoo Finance, save to CSV, and upsert data into Pinecone.
    """
    try:
        # Fetch data from Yahoo Finance
        stock_data = yf.download(symbol, start=start_date, end=end_date)

        # Save data to CSV
        csv_file_path = os.path.join(folder_path, f'{symbol}.csv')
        stock_data.to_csv(csv_file_path)
        logger.info(f"Data for {symbol} saved to {csv_file_path}")

        # Load CSV data
        df = pd.read_csv(csv_file_path)

        # Select necessary columns and drop NaN values
        required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        if not all(col in df.columns for col in required_columns):
            logger.error(f"{csv_file_path} is missing one or more required columns: {required_columns}")
            return

        df = df[required_columns].dropna()

        # Convert each row into a single string of concatenated data
        text_data = df.apply(
            lambda row: f"Date: {row['Date']}, Open: {row['Open']}, High: {row['High']}, Low: {row['Low']}, Close: {row['Close']}, Adj Close: {row['Adj Close']}, Volume: {row['Volume']}",
            axis=1
        )

        # Generate embeddings for the stock data
        embeddings = embedding_model.encode(text_data.tolist(), convert_to_numpy=True)

        # Prepare data for Pinecone upsert
        vectors = [(f"{symbol}_{i}", embeddings[i].tolist(), {'symbol': symbol}) for i in range(len(embeddings))]

        # Upsert data into Pinecone index
        try:
            response = index.upsert(vectors)
            logger.info(f"Successfully upserted {len(vectors)} vectors for {symbol}.")
            logger.info(f"Pinecone response: {response}")
        except Exception as e:
            logger.error(f"Error upserting data into Pinecone for {symbol}: {e}")

        # Save the processed DataFrame to a new CSV file in the output folder
        processed_file_path = os.path.join(output_folder, f'processed_{symbol}.csv')
        df.to_csv(processed_file_path, index=False)
        logger.info(f"Processed data saved to {processed_file_path}")

    except Exception as e:
        logger.error(f"Error processing data for {symbol}: {e}")

In [12]:
# Loop through all symbols and process each
for symbol in symbols:
    fetch_and_save_stock_data(symbol)

[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for NVDA saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\NVDA.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for NVDA: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:04:31 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '6244', 'x-pinecone-request-id': '6608081258971590932', 'x-envoy-upstream-service-time': '15', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_NVDA.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for INTC saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\INTC.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for INTC: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:05:40 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '13529', 'x-pinecone-request-id': '5894802343651214187', 'x-envoy-upstream-service-time': '57', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_INTC.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for PLTR saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\PLTR.csv


...

ERROR:__main__:Error upserting data into Pinecone for CMG: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:36:29 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '763', 'x-pinecone-request-id': '8220050840944410578', 'x-envoy-upstream-service-time': '14', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_CMG.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for BCS saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\BCS.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for BCS: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:37:06 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1419', 'x-pinecone-request-id': '2383404532506540172', 'x-envoy-upstream-service-time': '15', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_BCS.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for UAA saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\UAA.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for UAA: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:37:43 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1089', 'x-pinecone-request-id': '385965260636459097', 'x-envoy-upstream-service-time': '16', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_UAA.csv


In [11]:
csv_files = ['NVDA.csv', 'INTC.csv', 'PLTR.csv', 'TSLA.csv', 'AAPL.csv', 'BBD.csv', 'T.csv', 'SOFI.csv',
             'WBD.csv', 'SNAP.csv', 'NIO.csv', 'BTG.csv', 'F.csv', 'AAL.csv', 'NOK.csv', 'BAC.csv',
             'CCL.csv', 'ORCL.csv', 'AMD.csv', 'PFE.csv', 'KGC.csv', 'MARA.csv', 'SLB.csv', 'NU.csv',
             'MPW.csv', 'MU.csv', 'LCID.csv', 'NCLH.csv', 'RIG.csv', 'AMZN.csv', 'ABEV.csv', 'U.csv',
             'LUMN.csv', 'AGNC.csv', 'VZ.csv', 'WBA.csv', 'WFC.csv', 'RIVN.csv', 'UPST.csv', 'GRAB.csv',
             'CSCO.csv', 'VALE.csv', 'AVGO.csv', 'PBR.csv', 'GOOGL.csv', 'SMMT.csv', 'GOLD.csv',
             'CMG.csv', 'BCS.csv', 'UAA.csv']

In [12]:
# Kiểm tra các tệp có trong thư mục
csv_files_in_folder = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Lấy các mã từ tên các file CSV
csv_symbols_in_folder = [f.split('.')[0] for f in csv_files_in_folder]

# Kiểm tra các mã cổ phiếu nào thiếu
missing_symbols = [symbol for symbol in symbols if symbol not in csv_symbols_in_folder]

# In ra kết quả
if missing_symbols:
    print(f"Các mã cổ phiếu bị thiếu: {missing_symbols}")
else:
    print("Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.")

Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.


# SVR Model

In [24]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import json

In [18]:
# Hàm load dữ liệu
def load_stock_data(symbol):
    file_path = os.path.join(folder_path, f'{symbol}.csv')
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df = df[['Date', 'Close']]  # Giữ lại cột Date và Close
    return df

In [19]:
# Hàm chuẩn bị dữ liệu cho SVR
def prepare_svr_data(df, time_step=60):
    data = df['Close'].values
    data = data.reshape(-1, 1)

    # Chuẩn hóa dữ liệu với MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    x_data, y_data = [], []
    for i in range(time_step, len(scaled_data)):
        x_data.append(scaled_data[i-time_step:i, 0])
        y_data.append(scaled_data[i, 0])

    x_data, y_data = np.array(x_data), np.array(y_data)

    return x_data, y_data, scaler

In [20]:
# Hàm huấn luyện SVR
def train_svr(x_train, y_train):
    svr_model = SVR(kernel='rbf', C=1e3, gamma=0.1)
    svr_model.fit(x_train, y_train)
    return svr_model

# Hàm dự báo với SVR
def predict_with_svr(model, x_test, scaler):
    predicted_scaled = model.predict(x_test)
    predicted_prices = scaler.inverse_transform(predicted_scaled.reshape(-1, 1))
    return predicted_prices

In [21]:
# Hàm dự báo tương lai (365 ngày)
def forecast_future_with_svr(model, last_sequence, scaler, days=365):
    future_prices = []
    for _ in range(days):
        next_price_scaled = model.predict(last_sequence.reshape(1, -1))
        next_price = scaler.inverse_transform(next_price_scaled.reshape(-1, 1))
        future_prices.append(next_price[0, 0])

        # Cập nhật dãy cuối cùng với giá dự báo để tiếp tục dự báo
        last_sequence = np.append(last_sequence[1:], next_price_scaled)

    return future_prices

In [25]:
# List to store forecast results for all symbols
forecast_results = []

for symbol in symbols:
    print(f"Processing {symbol}...")

    # Load data
    try:
        df = load_stock_data(symbol)
    except FileNotFoundError:
        print(f"Error: CSV file for symbol '{symbol}' not found in {folder_path}. Skipping.")
        continue
    except Exception as e:
        print(f"Error loading data for symbol '{symbol}': {e}. Skipping.")
        continue

    # Ensure there is enough data
    if len(df) < 61:  # At least time_step + 1
        print(f"Not enough data for symbol '{symbol}'. Skipping.")
        continue

    # Prepare data
    x_data, y_data, scaler = prepare_svr_data(df)

    # Split into training and testing sets (80% train, 20% test)
    split_index = int(len(x_data) * 0.8)
    x_train, y_train = x_data[:split_index], y_data[:split_index]
    x_test, y_test = x_data[split_index:], y_data[split_index:]

    # Inverse transform y_train for storing actual prices
    y_train_original = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten().tolist()

    # Train SVR model
    svr_model = train_svr(x_train, y_train)

    # Predict on test set
    predicted_prices = predict_with_svr(svr_model, x_test, scaler).flatten().tolist()

    # Inverse transform y_test for evaluation
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten().tolist()

    # Calculate evaluation metrics
    mse = mean_squared_error(y_test_original, predicted_prices)
    rmse = math.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test_original, predicted_prices)

    print(f"Completed processing for {symbol}. RMSE: {rmse:.4f}, MSE: {mse:.4f}, MAPE: {mape:.4f}")

    # Forecast future prices (365 days)
    last_sequence = x_test[-1]
    future_prices = forecast_future_with_svr(svr_model, last_sequence, scaler, days=365)

    # Create future dates starting from the day after the last date in the dataset
    future_dates = pd.date_range(start=df['Date'].max() + pd.Timedelta(days=1), periods=365, freq='D')
    future_df = pd.DataFrame({
        'Date': future_dates,
        'Predicted_Price': future_prices
    })

    # Optionally, save future predictions per symbol as separate CSVs
    # Uncomment the following lines if you want to save individual future predictions
    # future_output_file = os.path.join(output_folder, f'{symbol}_future_predictions.csv')
    # future_df.to_csv(future_output_file, index=False)

    # Append results to the forecast_results list
    forecast_results.append({
        'Symbol': symbol,
        'RMSE': rmse,
        'MSE': mse,
        'MAPE': mape,
        'Train_Prices': y_train_original,
        'Test_Prices': y_test_original,
        'Predicted_Prices': predicted_prices,
        'Future_Price_Predictions': future_prices
    })

# ============================
# Saving the Forecast Summary
# ============================

# Check if any results were collected
if not forecast_results:
    print("No forecast results to save. Exiting.")
else:
    # Convert the list of dictionaries to a DataFrame
    df_summary = pd.DataFrame(forecast_results)

    # Convert lists to JSON strings for CSV compatibility
    list_columns = ['Train_Prices', 'Test_Prices', 'Predicted_Prices', 'Future_Price_Predictions']
    for col in list_columns:
        df_summary[col] = df_summary[col].apply(lambda x: json.dumps(x))

    # Define the output file path
    output_file = os.path.join(output_folder, 'forecast_summary_svr.csv')

    # Save the DataFrame to a CSV file
    df_summary.to_csv(output_file, index=False)

    print(f"\nForecast summary has been saved to {output_file}")

Processing NVDA...
Completed processing for NVDA. RMSE: 50.1171, MSE: 2511.7245, MAPE: 0.5049
Processing INTC...
Completed processing for INTC. RMSE: 3.6036, MSE: 12.9861, MAPE: 0.0979
Processing PLTR...
Completed processing for PLTR. RMSE: 2.3759, MSE: 5.6449, MAPE: 0.0781
Processing TSLA...
Completed processing for TSLA. RMSE: 22.3354, MSE: 498.8716, MAPE: 0.1015
Processing AAPL...
Completed processing for AAPL. RMSE: 31.4207, MSE: 987.2615, MAPE: 0.1262
Processing BBD...
Completed processing for BBD. RMSE: 0.6636, MSE: 0.4404, MAPE: 0.2275
Processing T...
Completed processing for T. RMSE: 1.4779, MSE: 2.1843, MAPE: 0.0725
Processing SOFI...
Completed processing for SOFI. RMSE: 0.4085, MSE: 0.1669, MAPE: 0.0433
Processing WBD...
Completed processing for WBD. RMSE: 8.0455, MSE: 64.7301, MAPE: 0.7906
Processing SNAP...
Completed processing for SNAP. RMSE: 5.0843, MSE: 25.8504, MAPE: 0.4298
Processing NIO...
Completed processing for NIO. RMSE: 3.7698, MSE: 14.2110, MAPE: 0.6423
Processi

In [26]:
# Chuyển kết quả thành DataFrame để dễ dàng truy vấn
summary_svr = pd.DataFrame(forecast_results)

In [27]:
summary_svr

,Symbol,RMSE,MSE,MAPE,Train_Prices,Test_Prices,Predicted_Prices,Future_Price_Predictions
0,NVDA,50.117108,2511.724528,0.504938,"[0.4907500147819519, 0.48925000429153437, 0.48...","[12.51200008392334, 13.166999816894531, 13.208...","[17.572944295730203, 17.55778906431745, 17.548...","[17.64174288667546, 17.671253202996457, 17.693..."
1,INTC,3.603622,12.986092,0.097906,"[36.22999954223633, 35.91999816894531, 35.5600...","[26.969999313354492, 27.70000076293945, 27.639...","[30.39895209666215, 30.21473516823568, 30.2661...","[29.034137172590395, 29.529253790267123, 30.02..."
2,PLTR,2.375905,5.644923,0.078135,"[27.750000000000004, 25.6299991607666, 24.6599...","[17.840000152587887, 17.950000762939453, 17.25...","[19.127490887495462, 19.053047884982778, 18.95...","[30.03555515514592, 29.06110509791109, 28.3284..."
3,TSLA,22.335433,498.871567,0.101496,"[13.800000190734865, 13.602666854858398, 13.18...","[242.3999938964844, 249.44000244140625, 240.80...","[294.48267879837334, 286.0461068797887, 280.74...","[250.1633322010434, 256.92304715059703, 263.78..."
4,AAPL,31.420717,987.261454,0.126236,"[27.4325008392334, 27.0575008392334, 26.6875, ...","[142.44999694824216, 146.1000061035156, 146.39...","[154.8348570638322, 154.45210635121862, 154.29...","[142.19716814335985, 141.40105063624713, 140.5..."
5,BBD,0.663599,0.440363,0.227539,"[8.836088180541994, 8.443526268005371, 8.26446...","[4.059999942779541, 4.090000152587891, 4.11999...","[4.110432971465953, 4.184916093703884, 4.21576...","[3.528078594331514, 3.591876835035827, 3.63660..."
6,T,1.477944,2.184318,0.072450,"[24.29002952575684, 24.35800552368164, 24.2749...","[15.899999618530272, 16.09000015258789, 15.930...","[17.259944636599027, 17.297251467293098, 17.31...","[20.434361382638333, 20.32068516612367, 20.284..."
7,SOFI,0.408481,0.166857,0.043310,"[17.14999961853027, 17.270000457763672, 16.719...","[8.529999732971191, 8.319999694824219, 8.47999...","[8.442371330863613, 8.439307593182903, 8.40138...","[7.496727789138786, 7.4764900462553765, 7.4746..."
8,WBD,8.045505,64.730144,0.790618,"[33.380001068115234, 32.869998931884766, 33.18...","[11.949999809265137, 12.359999656677246, 12.22...","[18.124527680762686, 18.159140570415605, 18.19...","[18.987035511934465, 19.568905948196193, 20.09..."
9,SNAP,5.084326,25.850371,0.429798,"[21.21999931335449, 21.450000762939453, 21.209...","[11.210000038146973, 11.239999771118164, 11.00...","[16.120260711424937, 16.16266028575607, 16.182...","[12.530394908053768, 13.13340357675532, 13.765..."


In [28]:
import os
import pandas as pd
import numpy as np
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import logging
import json
from datetime import timedelta

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# Paths to data
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'  # Folder with stock CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'  # Output folder

# List of stock symbols
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
           'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
           'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
           'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
           'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
           'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
           'CMG', 'BCS', 'UAA']

# Load forecast_summary_svr.csv
forecast_summary_file = os.path.join(output_folder, 'forecast_summary_svr.csv')
if os.path.exists(forecast_summary_file):
    forecast_summary_df = pd.read_csv(forecast_summary_file)
    # Parse JSON-serialized lists into actual lists
    list_columns = ['Predicted_Prices', 'Test_Prices', 'Future_Price_Predictions', 'Train_Prices']
    for col in list_columns:
        if col in forecast_summary_df.columns:
            forecast_summary_df[col] = forecast_summary_df[col].apply(
                lambda x: json.loads(x) if pd.notnull(x) else []
            )
    logging.info("Loaded 'forecast_summary_svr.csv' successfully.")
else:
    forecast_summary_df = pd.DataFrame()
    logging.warning("Could not find 'forecast_summary_svr.csv'. Evaluation metrics and forecasts will not be displayed.")

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server  # For deployment

# Define app layout
app.layout = html.Div([
    html.H1('SVR Stock Price Predictions Dashboard'),
    html.Div([
        html.Label('Select Stock Symbol'),
        dcc.Dropdown(
            id='stock-dropdown',
            options=[{'label': s, 'value': s} for s in symbols],
            value='NVDA'  # Default stock
        )
    ], style={'width': '25%', 'display': 'inline-block'}),
    html.Div([
        html.Label('Select Date Range'),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed='2014-09-18',  # Adjust based on your data
            max_date_allowed='2024-09-18',  # Adjust based on your data
            start_date='2014-09-18',        # Default start date
            end_date='2024-09-18'           # Default end date
        )
    ], style={'display': 'inline-block', 'marginLeft': '50px'}),
    html.Div([
        html.Label('Show 1-Year Forecast'),
        dcc.Checklist(
            id='forecast-checkbox',
            options=[{'label': 'Include 1-Year Forecast', 'value': 'show_forecast'}],
            value=[],
            inline=True
        )
    ], style={'marginTop': '20px'}),
    dcc.Graph(id='price-graph'),
    html.Div(id='metrics-output', style={'marginTop': '20px'})
])

# Define callback to update graph and evaluation metrics
@app.callback(
    [Output('price-graph', 'figure'),
     Output('metrics-output', 'children')],
    [Input('stock-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date'),
     Input('forecast-checkbox', 'value')]
)
def update_graph(selected_stock, start_date, end_date, forecast_option):
    # Convert start_date and end_date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Get the forecast data for the selected stock
    forecast_row = forecast_summary_df[forecast_summary_df['Symbol'] == selected_stock]

    if forecast_row.empty:
        logging.error(f"No forecast data found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"No forecast data found for {selected_stock}."

    # Parse 'Test_Prices', 'Predicted_Prices', 'Future_Price_Predictions', 'Train_Prices'
    test_prices = forecast_row.iloc[0]['Test_Prices']
    predicted_prices = forecast_row.iloc[0]['Predicted_Prices']
    future_prices = forecast_row.iloc[0]['Future_Price_Predictions']
    train_prices = forecast_row.iloc[0]['Train_Prices']

    # Load the stock data to get dates
    stock_data_file = os.path.join(folder_path, f'{selected_stock}.csv')
    if not os.path.exists(stock_data_file):
        logging.error(f"Stock data file not found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Stock data file not found for {selected_stock}."

    df_stock = pd.read_csv(stock_data_file)
    if 'Date' not in df_stock.columns or 'Close' not in df_stock.columns:
        logging.error(f"Incorrect data format in stock data file for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Incorrect data format in stock data file for {selected_stock}."

    # Convert 'Date' to datetime
    df_stock['Date'] = pd.to_datetime(df_stock['Date'])
    dates = df_stock['Date']
    data_length = len(df_stock)

    time_step = 60  # As used in your model
    total_samples = data_length - time_step

    # Calculate train_size and test_size based on 80:20 split
    train_size = int(total_samples * 0.8)
    test_size = total_samples - train_size

    # Ensure that the length of 'Test_Prices' matches 'test_size'
    if len(test_prices) != test_size or len(predicted_prices) != test_size:
        logging.error(f"Length mismatch between Test/Predict prices and test set for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Length mismatch in data for {selected_stock}."

    # Get indices for train and test sets in original data
    train_indices_in_y_data = range(0, train_size)
    test_indices_in_y_data = range(train_size, train_size + test_size)
    indices_train = [i + time_step for i in train_indices_in_y_data]
    indices_test = [i + time_step for i in test_indices_in_y_data]

    # Ensure indices do not exceed the length of dates
    indices_train = [i for i in indices_train if i < len(dates)]
    indices_test = [i for i in indices_test if i < len(dates)]

    # Dates for train and test sets
    dates_train = dates.iloc[indices_train].reset_index(drop=True)
    dates_test = dates.iloc[indices_test].reset_index(drop=True)

    # Create DataFrame for train set
    df_train = pd.DataFrame({
        'Date': dates_train,
        'Train_Price': train_prices
    })

    # Create DataFrame for test set
    df_test = pd.DataFrame({
        'Date': dates_test,
        'Test_Price': test_prices,
        'Predicted_Price': predicted_prices
    })

    # Filter df_train and df_test based on 'start_date' and 'end_date'
    df_train_filtered = df_train[(df_train['Date'] >= start_date) & (df_train['Date'] <= end_date)]
    df_test_filtered = df_test[(df_test['Date'] >= start_date) & (df_test['Date'] <= end_date)]

    # Initialize data list for plotting
    data = []

    # Add Train Prices if available in the selected date range
    if not df_train_filtered.empty:
        data.append(
            go.Scatter(
                x=df_train_filtered['Date'],
                y=df_train_filtered['Train_Price'],
                mode='lines',
                name='Train Price',
                line=dict(color='red')
            )
        )

    # Add Actual Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Test_Price'],
                mode='lines',
                name='Actual Test Price',
                line=dict(color='green')
            )
        )

    # Add Predicted Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Predicted_Price'],
                mode='lines',
                name='Predicted Test Price',
                line=dict(color='blue')
            )
        )

    # Add future price predictions if selected
    if 'show_forecast' in forecast_option:
        # Generate future dates
        last_date = dates.iloc[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(future_prices), freq='D')

        if len(future_prices) != len(future_dates):
            logging.error(f"Length mismatch between future prices and future dates for {selected_stock}.")
        else:
            data.append(
                go.Scatter(
                    x=future_dates,
                    y=future_prices,
                    mode='lines',
                    name='1-Year Forecast',
                    line=dict(dash='dash', color='orange')
                )
            )

    # Create figure
    figure = {
        'data': data,
        'layout': go.Layout(
            title=f'{selected_stock} Price Prediction',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Price'},
            hovermode='closest'
        )
    }

    # Get RMSE, MSE, and MAPE
    try:
        rmse = float(forecast_row.iloc[0]['RMSE'])
        mse = float(forecast_row.iloc[0]['MSE'])
        mape = float(forecast_row.iloc[0]['MAPE'])
    except (ValueError, TypeError) as e:
        logging.error(f"Error parsing evaluation metrics for {selected_stock}: {e}")
        rmse = mse = mape = None

    # Prepare metrics output
    if rmse is not None and mse is not None and mape is not None:
        metrics_output = [
            html.P(f'Root Mean Squared Error (RMSE): {rmse:.4f}'),
            html.P(f'Mean Squared Error (MSE): {mse:.4f}'),
            html.P(f'Mean Absolute Percentage Error (MAPE): {mape:.2%}')
        ]
    else:
        metrics_output = [html.P("Evaluation metrics are not available.")]

    return figure, metrics_output

# Run Dash app
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)

INFO:root:Loaded 'forecast_summary_svr.csv' successfully.
